## Generate LMDB by batches
First we will test the functionality in place and check it works for a small amount of batches before generating the whole DB.

In [29]:
from kipoi_cadd.data import CaddBatchDataset, cadd_serialize_numpy_row
from kipoi_cadd.data_utils import dir_batch_generator, OrderedDict, get_one_batch
from kipoi_cadd.utils import dump_to_pickle, load_pickle
import time
import pyarrow as pa
import lmdb
from tqdm import tqdm
import blosc
import pandas as pd
import numpy as np

In [2]:
training_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
lmdb_dir = training_dir + "lmdb"
lmdb_batch_dir = training_dir + "lmdb_batched"
csv_file = training_dir + "training_data.imputed.csv"
valid_id_file = training_dir + "valid_idx.pkl"
all_ids_file = training_dir + "variant_ids.pkl"
ids_10k_file = training_dir + "sample_indices/ids_10k.pkl"
std_scaler_file = training_dir + "stats/standard_scaler_first10k.pkl"
short_csv = training_dir + "last10k.csv"
batch_idx_file = training_dir + "shuffle_splits/batch_idxs_256.pkl"

In [44]:
np.iinfo(np.int32).max

2147483647

In [63]:
def create_lmdb_from_iterator(it, lmdb_batch_dir, variant_ids_file, num_batches=-1,
                              map_size=23399354270):
    start = time.time()

    index_mapping = OrderedDict()
    map_size = None
    txn = None
    batch_num = 0
    variant_ids = load_pickle(variant_ids_file)

    env = lmdb.Environment(lmdb_batch_dir, map_size=map_size, max_dbs=0, lock=False)
    with env.begin(write=True, buffers=True) as txn:
        for batch in tqdm(it, total=num_batches):
            b = {
                "batch_id": np.int32(batch_num),
                "inputs": batch[0].values.astype(np.float16),
                "targets": batch[1].values.astype(np.float16),
                "metadata": {
                    "row_num": np.array(batch[0].index, dtype=np.int32),
                    "variant_id": np.array(variant_ids.loc[batch[0].index], dtype='<U20')
                }
            }

            # Serialize and compress
            buff = pa.serialize(b).to_buffer()
            blzpacked = blosc.compress(buff, typesize=8, cname='blosclz')

            try:
                txn.put(str(batch_num).encode('ascii'), blzpacked)
            except lmdb.MapFullError as err:
                print(str(err) + ". Exiting the program.")

            batch_num += 1
            if batch_num >= num_batches: break

    print("Finished putting " + str(batch_num) + " batches to lmdb.")
    end = time.time()
    print("Total elapsed time: {:.2f} minutes.".format(
        (end - start) / 60))

def calculate_map_size(row_example, nrows, multiplier=1.9):
    row_size = pa.serialize(row_example).to_buffer().size
    map_size = int(row_size * nrows * multiplier)
    return map_size

In [56]:
it = dir_batch_generator(training_dir + "shuffle_splits/training/", 256)
test_batch = next(it)
variant_ids = load_pickle(all_ids_file)
nrows = len(variant_ids)

In [59]:
row_example = {
    "batch_id": np.int32(0),
    "inputs": test_batch[0].values.astype(np.float16),
    "targets": test_batch[1].values.astype(np.float16),
    "metadata": {
        "row_num": np.array(test_batch[0].index, dtype=np.int32),
        "variant_id": np.array(variant_ids.loc[test_batch[0].index], dtype='<U20')
        }
    }

In [72]:
ms = calculate_map_size(row_example, 3)
it = dir_batch_generator(training_dir + "shuffle_splits/training/", 256)
create_lmdb_from_iterator(it, lmdb_batch_dir, all_ids_file, num_batches=3, map_size=ms)

 33%|███▎      | 1/3 [00:07<00:14,  7.42s/it]


Finished putting 3 batches to lmdb.
Total elapsed time: 0.29 minutes.


In [68]:
batch = get_one_batch(lmdb_batch_dir, 2)

In [71]:
batch['inputs'].shape

(256, 1063)

In [7]:
aja = load_pickle(batch_idx_file)

In [28]:
np.unique(batch['metadata']['variant_id'] == np.array(list(aja[2]["variant_ids"]), dtype='<U20'))

array([ True])

In [12]:
aja[0]

{'row_nrs': <kipoi_cadd.utils.OrderedSet at 0x7f3d64ed5160>,
 'variant_ids': <kipoi_cadd.utils.OrderedSet at 0x7f3d64ed54e0>}

In [ ]:
CaddBatchDataset(lmdb_dir, batch_idx_file)